## Введение в нейронные сети.

XOR problem:

- однослойным перцептроном (это по сути просто логрег на несколько классов) не реализовать XOR, но если добавить еще один признак - произведение признаков, то получится
- Или xor реализется использованием 2 слоев

Интерес к нейронкам угас в нулевых, потому что был придуман градиентный бустинг, который почти не переобучается

### Многоклассовый логрег

О логистической регрессии (для многоклассовой классификации на m классов): 
- Перед выходом в нейронной сети применяется софтмакс для получения вектора вероятностей, а из него уже можно класс выбирать

- Мы хотим, чтобы logreg возвращал нам вектор из $P(y=k|x)$ - вероятностей того, что наш объект принадлежит к k классу, это мы будем делать, положив предсказательную функцию (или функцию активации, если рассматривать логрег, как однослойную нейронку) в виде
$$
softmax( logits(\vec{x}) ) - \text{вектор из вероятностей}
$$

- По сути логрег строит гиперплоскости, разделяющие всю выборку, а вероятность принадлежности к классу - тоже связана с расстояниями до каждой плоскости:
$$
P(y=k| \vec{x}) = \dfrac{exp(logits(\vec{x})_k)}{\sum_{i=1}^m exp(logits(\vec{x})_i)} \text{( - k-ая компонента softmax(logits(x)))}
$$

Для подбора весов в логитах используется лосс - logloss - это минус логарифм правдоподобия (у нас есть вектор весов + свободный член для каждого класса, в случае бинарной классификации нам хвтает лишь одного вектора весов+свобдного члена, потому что P(0|x)=1-P(1|x) )

И в обычном логреге мы просто градиентным спуском находим минимум logloss и каждый шаг обновляем все вектора весов + своб чл

- С точки зрения слоя нейронной сети:

W - это матрица из векторов весов для каждого класса, b - вектор из св чл для каждого класса, то есть на втором этапе мы получаем вектор из логитов, XW - вектор из вероятностей, его длина - количество классов. (X - строка)

Пусть X имеет 30 признаков, а классов у нас 50, тогда: у W столбцы - это вектора весов для каждого класса, то есть W: 30x50

$$
XW=(\vec{w}_1\cdot X,...,\vec{w}_m\cdot X)^T, \text{ здесь } \vec{w}_k \text{- это целый вектор весов для k класса}
$$

$$
\vec{X}---> XW + b ---> (P(1|X),...,P(k|X))^T
$$

### Про нейронки

- А если вдруг выборка линейно не разделима? (можно использовать SVM и подобрать в нем ядро или ансамбли деревьев)

- SVM - это задача поиска разделяющей плоскости для двух классов так, чтобы было максимальным самое маленькое расстояние, то есть нам надо максимизировать $1/||w||$, если выборка линейно разделима, то тут все очевидно, а если нет, то надо подбирать ядра, то есть надо добавить какой-то новый столбец признаков так, чтобы выборка стала разделимой

- Нейронные сети теперь будут моделировать фичи за нас и только потом классифицировать, что имеется в виду:

- Мы к исходной выборке применяем линейную модель и потом к результату этой модели применяем функцию активации, таким образом нейронная сеть за нас подбирает что-то наподобие kernel для SVM, схемка:

x $\in X$ -->(Линейн преобр) x' $\in X'$ (у X' уже другая размерность) -->(применяем нелинейную ф активации) -->(Logreg) --> Output  

- Пример:

1) На вход подаем X - пространство признаков размерности 10, затем при помощи первой линейной модели переводим в $XW_1$ - получаем, например, пространство признаков размерности 50, 
   
2) Потом применяем функцию активации с каждому полученному признаку, т.к. иначе во всем этом нет смысла и мы просто строим одну линейную модель, потому что сумма линейных комбинаций - линейная комбинация, а так функции активации позволяют нам работать с любой выборкой, даже линейно неразделимой, то есть линейная модель + функция активации - это что-то типо настроемого ядра SVM

(Если не применять функции активации, то каскад линейных преобразований схлопнется просто в одно линейное преобразование):
$$
(XW + b)\hat{W} + \hat{b}=XW\hat{W} +b\hat{W}+\hat{b}
$$
   
3) И теперь мы имеем некоторое пространство размерности 50 с новыми признаками - для него теперь мы применяем логистическую регрессию: получаем вектор вероятностей принадлежности к каждому классу

4) Ну и теперь уже делаем как хотим - либо output - это наиболее вероятный класс либо просто весь вектор вероятностей

Теперь решающий алгоритм имеет вид длинной цепочки композиции линейных моделей и функций активации, которая завершается финальным логрегом, конечно же, наши градиентные методы оптимизации до сих пор работают и мы до сих пор можем оптимизировать наш Logloss для последней логрегрессии, но для этого надо побольше проивзодных посчитать

## Backpropagation

Сила нейронных сетей в дифференцируемости всего (главное доопределить так функции чтобы они 1 раз были диф-мы)

Функции активации: Они нужны для того, чтобы сделать некоторые нелинейности

1)сигмоида

2)tanhx

3)max(0, x) - ReLU 

- Понятия:

Layer - структурная единица в цепочке композиции функций, которые применяются ко входному объекту (линейный слой можно мыслить как афинное преобразование в афинном пространстве признаков) 

Функция активации - то, что всегда применяется к результату линейого слоя (афинного преобразования) перед ним 

Backpropagation - "fancy word for chain rule"

Идея в том, что нам все так же надо оптимизировать функцию потерь, однако теперь она явно не зависит от весов, которые мы подбираем для минимизации лосса.

Но функция потерь теперь зависит от значения функции активации, которая зависит от весов. Используя это нам надо получить производную лосса по весу.

- См pdf для подробных объяснений

- Backprop сразу на всех слоях позволяет менять веса

### Про функции активации подробнее:

Нейронная сеть с каждым шагом меняет признаковое представление объекта

1) Сигмоида
+ имеет конечкую область значений
+ неплохо интерпретируется
  
- Затухает на хвостах: довольно большие хвосты с околонулевым градиентом из-за чего модель на этих ховстах не будет практически обновлять веса
- нецентрированный выход => сдвиг постоянный
- cчитать сложно

2) tanh
+ как сигмоида, но еще и центрирована

3) ReLU
- легко считать производную
- градиент не затухает
- выход не центрирован
- умирает при отрицательном входе

4) Leaky ReLU = max(0.01x, x)

##### Функции активации подбирают под задачу

Если не знать какую юзать, то:
- Использовать ReLU
- не использовать sigmoid
- попробовать Leaky ReLU/ELU, tanh

## Нейронные сети

Cамая простая версия - это fully connected - это просто нейронка из линейных слоев

#### Fancy neural networks:

1) RNN (recurent)

Они умеют генерировать код/текст и тп по скормленным ей примерам

2) CNN

Похожи на то, как работает зрительная кора

У нейронной сети (пока) все должно быть дифференцируемо, поэтому не стоит линейный переход заменять на решающее дерево, иначе мы просто не сможем бэкпропнуть

#### Подробно про полносвязную нейронную сеть:

Модель нейрона:

Значение внутри нейрона это по-сути просто один из новых признаков в текущем слое, в нем лежит взвешенная суммы предыдущих признаков с весами, соответствующими этому нейрону, а на выход он передает функцию активации от этой взвешенной суммы

##### Многослойный перцептрон:

- Вся сеть состоит только из fully connected слоев, суть заключается просто в том, что мы на скрытых слоях постоянно переходим к новому признаковому описанию нашей выборки, а потом в конце делаем логистическую регрессию на выходе последнего скрытого слоя (или если задача классификации, то другое что-то делаем)
  
- полученный вектор softmax и будет вероятностями принадлежности к каждому классу

- Или можно мыслить это эквивалентно так: все скрытые слои - это переходы к новым признакам при помощи функции активации на взвешенной сумме, а для выходного слоя мы применяем softamx в качестве функции активации на предпоследнем слое(так как внутри нейроной у нас лежат взвешенные суммы, а они же и есть логиты для данного нейрона)

- Если задачу регрессии решаем, то просто в последнем слое решаем задачу регрессии на полученной выборке

! Но надо понимать, что мы не можем просто взять и перед выходным слоем заюзать какой-то недифференцируемый алгоритм, поскольку тогда мы просто не сможем посчитать производную и обновить веса, поэтому все устроенно так, как устроенно

В лекции записывают объект в столбец признаков, а весы нейрона в строчку, но мне вообще удобнее наоборот, но из контекста будет очевидно

Задача: подобрать матрицы весов и вектора сдвигов так, чтобы минимизировать лосс

- для классификации в качестве лосса просто logloss юзается обычно
- MSE для регресси можно юзать

https://youtu.be/O0nGKKFyYT4?t=1705 - для backprop